In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.signal import find_peaks
import time
import multiprocessing as mp
import pandas as pd
import copy
import sys

In [ ]:
#Class that save the groups of parameters that mutates in each Tournament
class parE:
    def _init_(self):
        self.param = [0]*34

#Class of the average of the subpopulations.
class PopAverage:
    k = 0
    n = 0
    Km = 0
    L = 0
    #period, amplitud and tau doesn't save yet.
    period = 0
    amp = 0
    tau = 0
    w = 0
    WCC = 0
    f = 0
    #Function that resets the values of genotype and phenotype
    def reset(self):
        self.k = 0
        self.n = 0
        self.Km = 0
        self.L = 0
        self.period = 0
        self.amp = 0
        self.tau = 0
        self.w = 0
        self.WCC = 0
        self.f = 0
    #Function that normalize the values of genotype and phenotype by f that contain the time per population size
    def normalize(self,N):
        f = self.f
        self.k /= f
        self.n /= f
        self.Km /= f
        self.L /= f
        self.period /= f
        self.amp /= f
        self.tau /= f
        self.w /= f
        self.WCC /= f
        self.f /= N
        

In [ ]:
#Classs that represent the network of each individual
class GeneNetwork:
    #Genotype parameters
    k = []
    n = []
    KM = []
    L = 0
    kd = []
    x = []
    LD=0
    #Tau, period and amplitude
    tau = 0
    period = 0
    amp = 0
    #Fitness parameters and omega is the convertion parameter of the concentrations to molecules
    v = 0.3
    omega = (14.1371*6.022*10**5)/43
    dawn = 1.5
    dusk = 11
    w = 0
    #Flags and parameters of heredity
    parent = 0
    mut = 0
    LTag = 0
    LMut = 0
    Lm = [0]*24
    LOsc = 0
    LFitm = 0
    LWCC = [0]*192
    LFit = [0]*192
    #Propensities
    r = [0]*39
    rT = 0
    #Stequiometric matrix
    V = [[1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,-1,-1,1,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,-1,1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,1,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,-1,1,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,-1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,-1,1,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,-1,1],
         [0,0,0,0,0,-1,1,0,0,0,0,0,0,0,0],
         [-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,-1,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,-1,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,-1,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,-1,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,-1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,-1,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,-1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,-1,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,-1,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,-1,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1]]
    #Time (time, t) and concentrations of the molecules (y)
    time = 0
    y=[]
    t=[0]
    
    #Function of the inicialization of parameters.
    def ic(self,x0,k0,L0,n0,KM0,Kd0,alg):
        self.x = np.array(x0,dtype=np.float64)
        self.k = k0[:]
        self.L = L0
        self.n = n0[:]
        self.KM = KM0[:]
        self.kd = Kd0[:]
        self.y = [[i] for i in self.x]
        if not alg:
            self.omega = 1
            self.sol = solve_ivp(self.ODE,[0,24],self.x,dense_output=True)
        self.x *= omega
        self.dawn *= omega
        self.dusk *= omega
        
    #Function to update the propensities
    def updateR(self):
        x = self.x
        k = self.k
        L = self.L
        n = self.n
        KM = self.KM
        kd = self.kd
        
        m = self.time%24
        if m>=12 or not self.LD:
            l = 0
        else:
            l = L
        
        self.r[0] = k[0]
        self.r[1] = k[1]*x[5] 
        self.r[2] = k[2]*x[6]
        self.r[3] = k[3]/(1 + KM[0]*x[5])
        self.r[4] = k[4]*x[11]
        self.r[5] = k[5]*x[0] 
        self.r[6] = k[6]*(x[0] + x[10] + KM[1] -  np.sqrt(pow(float(x[0] + x[10] + KM[1]),2) - 4*x[0]*x[10]))/2 
        self.r[7] = k[7]*x[1] 
        self.r[8] = k[8]*x[2]*x[3]
        self.r[9] = k[9]*x[4] 
        self.r[10] = k[10]*x[6] 
        self.r[11] = k[11]*x[5]*(pow(x[11],n[0])/(pow(x[11],n[0]) + pow(KM[2],n[0]))) 
        self.r[12] = k[12]*x[5]
        self.r[13] = k[13]*x[6]*(pow(x[14],n[1])/(pow(x[14],n[1]) + pow(KM[3],n[1]))) 
        self.r[14] = (k[14]*pow(float(KM[5]*x[5]),n[2]))/(pow(float(KM[4]*KM[5]),n[2]) + pow(float(KM[5]*x[5]),n[2]) + pow(float(KM[4]*x[6]),n[2])) 
        self.r[15] = (k[15]*pow(float(KM[4]*x[6]),n[2]))/(pow(float(KM[4]*KM[5]),n[2]) + pow(float(KM[5]*x[5]),n[2]) + pow(float(KM[4]*x[6]),n[2])) 
        self.r[16] = k[16]*x[7]
        self.r[17] = k[17]*x[8] 
        self.r[18] = k[18]*x[9] 
        self.r[19] = k[19]*x[10] 
        self.r[20] = k[20]*(pow(x[6],n[3])/(pow(KM[6],n[3]) + pow(x[6],n[3]))) 
        self.r[21] = k[21]*x[12]
        self.r[22] = k[22]*x[13]
        self.r[23] = l*x[6] 
        self.r[24] = kd[0]*x[0] 
        self.r[25] = kd[1]*x[1] 
        self.r[26] = kd[2]*x[2] 
        self.r[27] = kd[3]*x[3] 
        self.r[28] = kd[4]*x[4] 
        self.r[29] = kd[5]*x[5]
        self.r[30] = kd[6]*x[6] 
        self.r[31] = kd[7]*x[7] 
        self.r[32] = kd[8]*x[8]
        self.r[33] = kd[9]*x[9] 
        self.r[34] = kd[10]*x[10] 
        self.r[35] = kd[11]*x[11] 
        self.r[36] = kd[12]*x[12] 
        self.r[37] = kd[13]*x[13] 
        self.r[38] = kd[14]*x[14]
        
        self.rT = sum(self.r)
    
    #Function to calculate the time of stochastic models
    def updateTime(self):
        rand1 = np.random.rand()
        
        self.updateR()
        re_time = -np.log(1-rand1)/self.rT
        self.time += re_time
        if int(self.time*2)>int((self.time - re_time)*2):
            self.t.append((self.time//.5)*.5)
    
    #Function to select the reactions of the stochastic models
    def updateSystem(self):
        r = self.r
        rT = self.rT
        rand2 = np.random.rand()
        rR = sum(rand2>(np.cumsum(r)/rT))
        self.x += self.V[rR]
        if len(self.t)>len(self.y[0]):
            self.y = np.append(self.y,self.x.reshape(15,1),axis=1)
    
    #ODEs of the system
    def ODE(self,tm,y):
        def f_m(P_1,P_2,K_m):
            return (P_1 + P_2 + K_m -  np.sqrt(pow(float(P_1 + P_2 + K_m),2) - 4*P_1*P_2))/2
            
        wc1 = y[0]
        wc2 = y[1]
        WC1 = y[2]
        WC2 = y[3]
        WCCc = y[4]
        WCCn = y[5]
        laWCC = y[6]
        frq = y[7]
        FRQ = y[8]
        FFC = y[9]
        FFCp = y[10]
        FFCn = y[11]
        vvd = y[12]
        VVDc = y[13]
        VVDn = y[14]
        
        k=self.k
        L=self.L
        n=self.n
        K=self.KM
        kd=self.kd
        
        #Tiempo y periodos LD
        m=tm%24
        if m>=12 or not self.LD:
            l = 0
        else:
            l = L
    
        #Ecuaciones        
        dwc1 = k[0] + k[1]*WCCn + k[2]*laWCC - kd[0]*wc1
        dwc2 = k[3]/(1 + K[0]*WCCn) + k[4]*FFCn - kd[1]*wc2
        dWC1 = k[5]*wc1 + k[6]*f_m(wc1,FFCp,K[1]) - k[8]*WC1*WC2 - kd[2]*WC1
        dWC2 = k[7]*wc2 - kd[3]*WC2 - k[8]*WC1*WC2
        dWCCc = k[8]*WC1*WC2 - kd[4]*WCCc -  k[9]*WCCc
        dWCCn = k[9]*WCCc + k[10]*laWCC - kd[5]*WCCn - l*WCCn - k[12]*WCCn - k[11]*WCCn*(pow(FFCn,n[0])/(pow(K[2],n[0]) + pow(FFCn,n[0])))
        dlaWCC = L*WCCn + k[12]*WCCn - kd[6]*laWCC - k[10]*laWCC - k[13]*laWCC*(pow(VVDn,n[1])/(pow(K[3],n[1]) + pow(VVDn,n[1])))
        dfrq =  (k[14]*pow(float(K[5]*WCCn),n[2]) + k[15]*pow(float(K[4]*laWCC),n[2]))/(pow(float(K[4]*K[5]),n[2]) + pow(float(K[5]*WCCn),n[2]) + pow(float(K[4]*laWCC),n[2])) - kd[7]*frq
        dFRQ =  k[16]*frq - kd[8]*FRQ - k[17]*FRQ
        dFFC = k[17]*FRQ - kd[9]*FFC - k[18]*FFC
        dFFCp = k[18]*FFC - kd[10]*FFCp - k[19]*FFCp
        dFFCn = k[19]*FFCp - kd[11]*FFCn
        dvvd = (k[20]*pow(laWCC,n[3]))/(pow(K[6],n[3]) + pow(laWCC,n[3])) - kd[12]*vvd
        dVVDc = k[21]*vvd - kd[13]*VVDc -  k[22]*VVDc
        dVVDn = k[22]*VVDc - kd[14]*VVDn

        return [dwc1, dwc2, dWC1, dWC2, dWCCc, dWCCn, dlaWCC, dfrq, dFRQ, dFFC, dFFCp, dFFCn, dvvd, dVVDc, dVVDn]
    
    #Calculation of the fitness by lorentzian function
    def calculateFit(self):
        def Opt(time):
            return yd + amp*(np.cos(2*np.pi*(time+12)/24))
        WCC = self.x[5]
        time = self.time
        v = self.v
        amp = (self.dusk-self.dawn)/2
        yd = self.dawn + amp
        #WCCopt = np.array([Opt(time-2),Opt(time+2),Opt(time)-0.5,Opt(time)+0.5])
        #if WCC>min(WCCopt) and WCC<max(WCCopt):
        #    self.w=1
        #else:
        #    dif=min(abs(WCCopt-WCC))
        #    self.w =  1/(1+((dif)**2/(v*WCCopt[dif==abs(WCCopt-WCC)][0])))
        WCCopt=Opt(time)
        self.w =  1/(1+((WCC-WCCopt)**2/(v*WCCopt)))
    
    #Function of mutations
    def mutate(self,M,par_E):
        global k0, Km0
        
        randl = np.random.rand(35)
        qm = pow(M,(1-(2*randl)))
        qm *= par_E.param
        qm = np.where(qm>0,qm,1)
        
        self.k *= qm[0:23]
        self.n *= qm[23:27]
        self.KM *= qm[27:34]
        self.L *= qm[34]
        
        self.n[self.n>20] = 20
        self.k = np.where(self.k>(np.array(k0)*100),(np.array(k0)*100),self.k)
        self.KM = np.where(self.KM>(np.array(Km0)*100),(np.array(Km0)*100),self.KM)
        
        self.mut = 1
        self.LMut += 1

In [ ]:
#Function of the Tournament
def selectionTournament(myPop, N, st, u, M, par_E):
    global alg
    #Matrix with the groups of parameters that mutates togheter.
    mutscenarios = [[1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                    [0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0],
                    [0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0],
                    [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,1]]
    
    #Selection of the gruop of individuals that compete.
    indexT = np.random.randint(N,size=st)
    indexT = pd.unique(indexT)
    while len(indexT)<st:
        indexT = np.append(indexT,np.random.randint(N,size=(st-len(indexT))))
        indexT = pd.unique(indexT)
    #Selection of the best fitness individual.
    maxT = indexT[-1]
    for i in range(len(indexT)-1): 
        if myPop[indexT[i]].w>myPop[maxT].w:
            maxT=indexT[i]
    #Birth and death
    j = np.random.randint(N)
    myPop[j] = copy.deepcopy(myPop[maxT])
    myPop[j].parent = maxT
    #Choose if the new individual have a mutation
    if np.random.rand() <= u:
        #Choose the kind of mutation
        par_E.param = mutscenarios[np.random.randint(8)]
        #Mutate and recalculate the solution of deterministics models
        myPop[j].mutate(M,par_E)
        if not alg:
            myPop[j].sol = solve_ivp(myPop[j].ODE,[0,24],myPop[j].x,dense_output=True)

In [ ]:
#Function of each individual simulation at the next event of birth and death
def Simulations(Ind):
    #Global variables that are need
    global myPop
    global myPopT
    global alg
    global iG
    global tWait
    global tGlobal
    global tGen
    global tCycle
    global resultI
    global resultLxC
    global Cp
    global tCC
    #Stochastic simulations
    if alg:
        while myPop[Ind].time < iG:
            myPop[Ind].updateTime()
            myPop[Ind].updateSystem()
    #Deterministic simulation
    else:
        myPop[Ind].time = iG
        myPop[Ind].x = myPop[Ind].sol.sol(tCC).round(4)
        if ((iG-tWait)//0.5)*0.5<tGlobal:
            myPop[Ind].t = np.append(myPop[Ind].t,myPop[Ind].time.round(2))
            myPop[Ind].y = np.append(myPop[Ind].y,myPop[Ind].x.reshape(15,1),axis=1)
    #Calculate the fitness of each individual
    if (tCC>11 and tCC<14) or (tCC>22 or tCC<2):
        myPop[Ind].calculateFit()
        #Calculate the Fitness lineage
        myPop[Ind].LFitm += (myPop[Ind].w*tWait)
    else:
        myPop[Ind].w=0
    myPop[Ind].LFit[tGlobal//0.5] = myPop[Ind].w
    #Average parameters
    myPopT.k += np.array(np.array(myPop[Ind].k)*tWait, dtype = float)
    myPopT.Km += np.array(np.array(myPop[Ind].KM)*tWait, dtype = float)
    myPopT.L += myPop[Ind].L*tWait
    myPopT.n += np.array(np.array(myPop[Ind].n)*tWait, dtype = float)
    myPopT.w += myPop[Ind].w*tWait
    myPopT.period += myPop[Ind].period*tWait
    myPopT.amp += myPop[Ind].amp*tWait
    myPopT.tau += myPop[Ind].tau*tWait
    myPopT.WCC += myPop[Ind].x[5]*tWait
    myPopT.f += tWait
    
    
    #Each generation certain information are saved to the individual information and lineage per cycle
    if int(iG/TMAX)>tGen:
        #Its saved the individual information of 100 individuals
        if Ind in range(0,1000,10):
            resultI.append([tGen,Ind,myPop[Ind].k,myPop[Ind].L,myPop[Ind].KM,myPop[Ind].n,
                            #myPop[i].period,myPop[i].amp,myPop[i].tau,
                            myPop[Ind].w,myPop[Ind].y[5],myPop[Ind].t,myPop[Ind].parent])
        #Slope sign at each generation
        dif=myPop[Ind].y[5][0]-myPop[Ind].y[5][-1]
        if dif:
            myPop[Ind].Lm[int(tGen%24)]=dif/abs(dif)
        else:
            myPop[Ind].Lm[int(tGen%24)]=0
        #Re-start the concentration and time of the individuals at each generation
        myPop[Ind].y = [[i] for i in myPop[Ind].x]
        myPop[Ind].t = [myPop[Ind].time]
        #Lineage per cycle information
        if ((tGen+1)%Cp)==0:
            #Kind of oscilation of the lineage
            if (Osc==myPop[Ind].Lm).all():
                myPop[Ind].LOsc=0
            elif sum(myPop[Ind].Lm)==0:
                myPop[Ind].LOsc=1
            else:
                myPop[Ind].LOsc=2
                
            
            #Save Lineage per cycle information
            resultLxC.append([tGen+1,myPop[Ind].LTag,myPop[Ind].LMut,
                              myPop[Ind].LFitm/tCycle,myPop[Ind].LOsc,myPop[Ind].Lm,
                              myPop[Ind].k,myPop[Ind].L,myPop[Ind].KM,
                              myPop[Ind].n])
            #Re-start the flags of the lineage
            myPop[Ind].Lm = [0]*24
            myPop[Ind].LFit = 0
            myPop[Ind].LMut = 0
            myPop[Ind].LTag = Ind
            

In [ ]:
#ParameteresultIhe simulation
N = 5000 #Number of the population
st = 2000 #Selection parameter
u = 0 #mutation rate
M = 2 #mutation 
alg = 0 #type of simulation(Deterministic=0 and stochastic=1)
LD = 0 #type of environment(LD=0, DD=1)
#Omega selection to the initial conditions
if alg:
    omega = (14.1371*6.022*10**5)/43
else:
    omega = 1

TMAX = 2 #Time of generations
rRE = [0.3,0.4,0.5,0.6] #Rates of reproduction
GMAX = 2000  #Number of generations
Cp = 24 #Number of generations that complete a cycle
period = 24  #Period of optimal oscilation
#Initial parameters
x0 = [0.5356, 0.6377, 0.0224, 13.3137, 0.4153, 0.1235, 0, 0.0035, 0.0282, 0.0019, 0.0137, 0.0066, 0, 0, 0]
#k0 = [9.18236260e-01*omega, 1.31272389e+00, 1.05612953e+02, 1.51368699e+00*omega, 3.34960765e-02, 2.23445399e-01, 
#      2.12373377e+00, 2e+00, 4.72000000e-01/omega, 3.00000000e-01, 1.00000000e-03, 8.22563158e+00, 1.0e-03, 
#      1.08551588e+01, 3.49620056e+00, 3.43058956e+02*omega, 5.40000000e+00, 1.50e-01, 2e+00, 5e-02, 8.58342556e+02*omega, 
#      6.80000000e-01, 3.00000000e-01]
k0 = [1.19*omega,1.2,90,1.6*omega,0.03,0.226,0.8,1,0.472/omega,0.3,0.001,0.6,0.001,20,7.3,320*omega,0.19,
      0.15,0.1,0.1,800*omega,0.68,0.3]
L0 = 0.2
n0 = [4, 4, 4, 4]
#Km0 = [0.0271708/omega,  0.29283663*omega, 0.04372374*omega, 4.63129806*omega, 3.89618486*omega, 0.1634689*omega, 0.01747003]
Km0 = [0.03/omega,1*omega,0.475*omega,5*omega,0.1*omega,0.18*omega,0.02*omega]
kd0 = [2.4, 2.5, 0.135, 0.085, 0.05, 0.05, 6, 1, 0.69, 0.34, 0.34, 0.1, 6.2, 0.24, 0.24]
#Values of the slope of the perfect oscilation
Osc=np.append(np.ones(6),np.ones(6)*-1)
Osc=np.append(Osc,Osc)

par_E = parE()
numrep = 3
#Computational Time that iniciates the simulation
inicio = time.time()
print("Run Time: ",inicio)
sys.stdout.flush()
#Réplicas
for iS in range(numrep-1,numrep):
    #Inizializates the population
    myPop = [GeneNetwork() for i in range(0,N)]
    myPopT = PopAverage()
    #Variables that save the results
    resultAGS = []
    resultLxC = []
    resultI = []
    #Different times
    tGen = 0 #Number of generations
    tCycle = 0 #Number of time for each cycle
    tGlobal = 0 #Time to save concentrations
    iG = 0 #Time of the simulation
    
    #Inizializates the population with the initial parameters
    for i in range(0,N):
        randl = np.random.rand(35)
        qm = pow(M,(1-(2*randl)))
        
        k1 = k0*qm[0:23]
        n1 = n0*qm[23:27]
        Km1 = Km0*qm[27:34]
        L1 = L0*qm[34]
        
        myPop[i].ic(x0,k1,L1,n1,Km1,kd0,alg)
        myPop[i].LTag = i
        myPop[i].LD = LD
        myPop[i].period = period
    #Generaciones
    while iG < TMAX*GMAX:
        #linajes
        #Calculate the percentage of reproduction
        iR = int((iG%24)//6)
        rLE = int((N*rRE[iR])/TMAX)
        #Calculate the time of the events
        tWait = -np.log(1 - np.random.rand())/rLE
        #Update the times
        iG += tWait
        tGlobal = (iG//0.5)*0.5
        tCC = iG%24
        #Run the simulations
        for i in range(0,N):
            Simulations(i)
        #Time of the cycle to the normalization
        if (tCC>11 and tCC<14) or (tCC>22 or tCC<2):
            tCycle += tWait
        #For each generation its saved the average information
        if int(iG/TMAX)> tGen:
            #Update the generation time and normalize the population info
            tGen=iG//TMAX
            myPopT.normalize(N)
            resultAGS.append([tGen,myPopT.k,myPopT.L,myPopT.Km,myPopT.n,
                              #myPopT.period,myPopT.amp,myPopT.tau,
                              myPopT.WCC,myPopT.w])
            #Reset the Average info
            myPopT.reset()
            #Saves temporarily each cycle
            if tGen%Cp==0:
                tmpI = pd.DataFrame(resultI,columns=["Generation","Cell tag",
                                            "Expresion and repression","Constant of light rate",
                                            "Disociation Constants", "Hill exponents",
                                            #"Period","Amplitude","Tau",
                                            "Fitness","WCCn","Time","Parent tag"])
    
                tmpLxC = pd.DataFrame(resultLxC,columns=["Generation","LTag","LMut","LFit","LOsc","Lm",
                                                "Expresion and repression","Constant of light rate",
                                                "Disociation Constants", "Hill exponents"])
    
                tmpAGS = pd.DataFrame(resultAGS,columns=["Generation", "Expresion and repression",
                                                         "Constant of light rate","Disociation constants", 
                                                         "Hill exponents", #"Period","Amplitude","Tau",
                                                         "WCC","Fitness"])
    
                tmpI.to_csv("Result/Ind_inf_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
                tmpLxC.to_csv("Result/LineagexCycle_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
                tmpAGS.to_csv("Result/AveGenSub_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
                #Print that I utilize to know the time of the simulation
                print("iG: ",iG)
                #Reset the cycle time
                tCycle=0
                sys.stdout.flush()
        #Tournament selection each event
        selectionTournament(myPop, N, st, u, M, par_E)
        
    #Saves when a replicate finish
    resultI = pd.DataFrame(resultI,columns=["Generation","Cell tag",
                                            "Expresion and repression","Constant of light rate",
                                            "Disociation Constants", "Hill exponents",
                                            #"Period","Amplitude","Tau",
                                            "Fitness","WCCn","Time","Parent tag"])
    
    resultLxC = pd.DataFrame(resultLxC,columns=["Generation","LTag","LMut","LFit","LOsc","Lm",
                                            "Expresion and repression","Constant of light rate",
                                            "Disociation Constants", "Hill exponents"])
    
    resultAGS = pd.DataFrame(resultAGS,columns=["Generation","Expresion and repression constants",
                                                "Constant of light rate","Disociation constants", 
                                                "Hill exponents",#"Period","Amplitude","Tau",
                                                "WCC","Fitness"])
    
    resultI.to_csv("Result/Ind_inf_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
    resultLxC.to_csv("Result/LineagexCycle_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
    resultAGS.to_csv("Result/AveGenSub_u{}_M{}_st{}_N{}_v{}_alg{}_{}.csv".format(u,M,st,N,myPop[0].v,alg,iS))
    
#Print the time that are needed to complete the evolutionary simulation
fin = time.time()
print("Run Time: ",fin-inicio)

True